# How to Use the Web Services API for Dataset Searching
***

## Overview:   
The NASA Goddard Earth Sciences Data and Information Services Center (GES DISC) has developed an Application Program Interface (API) that is intended for users who prefer to employ our data search and subsetting services using scripts instead of the Web browser interface. The API protocol allows users to find the datasets and data granules they need, and to download any desired subsets. Information is passed back and forth in JavaScript Object Notation (JSON) format. 

## Example:  
This example code demonstrates how to use the API to search GES DISC data collections. The code prompts the user for a search string, submits the search to the GES DISC server, and then retrieves the search results. The response from the server contains a large collection of metadata for each dataset that matches the search string. We provide some examples below that will print out selected pieces of information for each item that might be relevant for a) narrowing down the search, b) finding documentation, or c) setting up another API script to perform a subsetting request. The final example shows how to write out the entire search response to a local file.

### Prerequisites:  
This example code is written in Python3 and requires these libraries: sys, json, datetime, urllib3, and certifi. 

### Procedure
The first step is to import the required Python libraries. If any of the following import commands fail, check the local Python environment and install any missing packages. 

In [2]:
import sys 
import json
import datetime
import certifi
import urllib3

Initialize the urllib PoolManager and set the base URL for dataset search API requests

In [3]:
# Create a PoolManager instance to make requests.
http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',ca_certs=certifi.where())

# Set the URL for the GES DISC API endpoint for dataset searches
url = 'https://disc.gsfc.nasa.gov/service/datasets/jsonwsp'
#url = 'https://uui-test.gesdisc.eosdis.nasa.gov/service/datasets/jsonwsp'

The user will be prompted for the desired search parameters -- the syntax is similar to what would be employed when searching with the Web browser interface. The search request is posted to the GES DISC server, and the response is checked for any errors. If there are errors, the user gets another prompt. When the results from a successful search have been retrieved, the script prints "OK", followed by the number of items in the search results.

Run the following block of code and try these four search strings, noting how the number of results diminishes with each additional keyword:  
_tropomi total column ozone  
s5p ozone  
sentinel-5p ozone L2_

In [13]:
# Prompt for search string keywords
# This will keep looping and prompting until search returns an error-free response
done = False
while done is False :
  myString=''
  while len(myString) < 1 : 
    myString = input("Enter search keywords: ")

  # Set up the JSON WSP request for API method: search
  search_request = {
    'methodname': 'search',
    'type': 'jsonwsp/request',
    'version': '1.0',
    'args': {'search': myString}
  }

  # Submit the search request to the GES DISC server
  hdrs = {'Content-Type': 'application/json',
          'Accept': 'application/json'}
  data = json.dumps(search_request)
  r = http.request('POST', url, body=data, headers=hdrs)
  response = json.loads(r.data)

  # Check for errors
  if response['type']=='jsonwsp/fault' :
    print('ERROR! Faulty request. Please try again.')
  else : 
    done = True
print('OK')

# Indicate the number of items in the search results
total = response['result']['totalResults']
if total == 0 :
    print('Zero items found')
elif total == 1 : 
    print('1 item found')
else :
    print('%d items found' % total)

Enter search keywords: GPM
OK
237 items found


The response from the server contains <b><em>all</em></b> the metadata for each item in the list of dataset search results. The entire response contains much more information than users will likely need. The four examples below illustrate how to extract and print out a few critical pieces of metadata for each search result item, which could prove helpful for narrowing down the dataset search or developing API subsetting scripts. Please see <em>"How to Use the Web Services API for Subsetting MERRA-2 Data"</em> for detailed instructions on subsetting with the API. 

The code below prints out the DatasetID (which is needed for any API subset requests) and the Label, a more human-readable description of the dataset. 

In [14]:
# Report on the results: DatasetID and Label
if total > 0 :
    for item in response['result']['items']:
        print('%-20s  %s' % (item['dataset']['id'], item['dataset']['label']))

GPM_1BGMI_07          GPM GMI Brightness Temperatures L1B 1.5 hours 13 km V07
GPM_1AGMI_07          GPM GMI unpacked data L1A 1.5 hours 13 km V07
GPM_BASEGPMGMI_07     GPM GMI Antenna Temperatures L1BASE 1.5 hours 13 km V07
GPM_2HSLH_07          GPM DPR Spectral Latent Heating Profiles L2 1.5 hours 5 km V07
GPM_2HCSH_07          GPM DPR and GMI Combined Stratiform Heating L2 1.5 hours 5 km V07
GPM_3GCSH_07          GPM DPR (Gridded Convective Stratiform Heating) L3 1.5 hours 0.5 degree x 0.5 degree V07
GPM_3HSLH_07          GPM DPR Spectral Latent Heating Profiles L3 1 month 0.5 degree x 0.5 degree V07
GPM_3GSLH_07          GPM DPR Gridded Orbital Spectral Latent Heating Profiles L3 1.5 hours 0.5 degree x 0.5 degree V07
GPM_3HCSH_07          GPM DPR and GMI Combined Convective Stratiform Heating L3 1 month 0.5 degree x 0.5 degree V07
GPM_BASEGPMGMI_RSS_07  GPM GMI RSS Common Calibrated Brightness Temperatures L1BASE 1.5 hours 13 km V07
GPM_BASEGPMGMI_XCAL_07  GPM GMI XCAL Common Calibr

The code below prints out the DatasetID along with the starting and ending dates. 

In [15]:
# Report on the results: DatasetID, StartDate, and EndDate
if total > 0 :
    for item in response['result']['items']:
        start = datetime.datetime.utcfromtimestamp(int(item['startDate']/1000))
        end   = datetime.datetime.utcfromtimestamp(int(item['endDate']/1000))
        print('%-20s  Start Date = %s    End Date = %s' % (item['dataset']['id'], start, end))                        

GPM_1BGMI_07          Start Date = 2014-03-04 00:00:00    End Date = 2022-12-21 23:59:59
GPM_1AGMI_07          Start Date = 2014-03-04 00:00:00    End Date = 2022-12-21 23:59:59
GPM_BASEGPMGMI_07     Start Date = 2014-03-04 00:00:00    End Date = 2022-12-21 23:59:59
GPM_2HSLH_07          Start Date = 2014-03-08 00:00:00    End Date = 2022-12-20 23:59:59
GPM_2HCSH_07          Start Date = 2014-03-08 00:00:00    End Date = 2022-12-20 23:59:59
GPM_3GCSH_07          Start Date = 2014-03-08 00:00:00    End Date = 2022-12-20 23:59:59
GPM_3HSLH_07          Start Date = 2014-03-01 00:00:00    End Date = 2022-11-30 23:59:59
GPM_3GSLH_07          Start Date = 2014-03-08 00:00:00    End Date = 2022-12-20 23:59:59
GPM_3HCSH_07          Start Date = 2014-03-01 00:00:00    End Date = 2022-11-30 23:59:59
GPM_BASEGPMGMI_RSS_07  Start Date = 2014-03-04 00:00:00    End Date = 2022-12-21 23:59:59
GPM_BASEGPMGMI_XCAL_07  Start Date = 2014-03-04 00:00:00    End Date = 2022-12-21 23:59:59
GPM_3HSLH_DAY_07  

The code below prints out the DatasetID and the Landing Page URL, which is the centralized access point for all the web-based information and services associated with that dataset.

In [16]:
# Report on the results: DatasetID and Landing Page URL
if total > 0 :
    for item in response['result']['items']:
        print('%-20s  %s' % (item['dataset']['id'], item['link'])) 

GPM_1BGMI_07          https://disc.gsfc.nasa.gov/datacollection/GPM_1BGMI_07.html
GPM_1AGMI_07          https://disc.gsfc.nasa.gov/datacollection/GPM_1AGMI_07.html
GPM_BASEGPMGMI_07     https://disc.gsfc.nasa.gov/datacollection/GPM_BASEGPMGMI_07.html
GPM_2HSLH_07          https://disc.gsfc.nasa.gov/datacollection/GPM_2HSLH_07.html
GPM_2HCSH_07          https://disc.gsfc.nasa.gov/datacollection/GPM_2HCSH_07.html
GPM_3GCSH_07          https://disc.gsfc.nasa.gov/datacollection/GPM_3GCSH_07.html
GPM_3HSLH_07          https://disc.gsfc.nasa.gov/datacollection/GPM_3HSLH_07.html
GPM_3GSLH_07          https://disc.gsfc.nasa.gov/datacollection/GPM_3GSLH_07.html
GPM_3HCSH_07          https://disc.gsfc.nasa.gov/datacollection/GPM_3HCSH_07.html
GPM_BASEGPMGMI_RSS_07  https://disc.gsfc.nasa.gov/datacollection/GPM_BASEGPMGMI_RSS_07.html
GPM_BASEGPMGMI_XCAL_07  https://disc.gsfc.nasa.gov/datacollection/GPM_BASEGPMGMI_XCAL_07.html
GPM_3HSLH_DAY_07      https://disc.gsfc.nasa.gov/datacollection/GPM_3HS

The code below prints out the DatasetID and a list of variable names and descriptions. The DatasetID and the variable name are pieces of metadata that are used for making subset requests with the API. 

In [17]:
# Report on the results: DatasetID and variable subsetting information
varSubset = False
for item in response['result']['items']:
    # Check for subset services
    if item['services']['subset']: 
        for ss in item['services']['subset']:
            # make sure variable subsetting is supported
            if 'variables' in ss['capabilities'] and 'dataFields' in ss :
                print('The %s service supports variable subsetting for %s' % 
                      (ss['agentConfig']['agentId'],item['dataset']['id']))
                print('Variable names are:')
                varSubset = True
                # Print a list of variable names and descriptions
                for var in ss['dataFields']:
                    print(var['value'])
                print()
if varSubset is False: 
    print('Variable subsetting is not available for %s' % item['dataset']['id'])


The OPeNDAP service supports variable subsetting for GPM_1BGMI_07
Variable names are:
S1_ScanTime_MilliSecond
S1_ScanTime_Minute
S1_ScanTime_Year
S1_ScanTime_SecondOfDay
S1_ScanTime_Second
S1_ScanTime_DayOfYear
S1_ScanTime_Hour
S1_ScanTime_Month
S1_ScanTime_DayOfMonth
S1_moonVectorInstFrame
S1_calCounts_hotLoadThermisterTemp
S1_calCounts_coldLoadReading
S1_calCounts_hotLoadnDiodeReading
S1_calCounts_hotLoadReading
S1_calCounts_coldLoadnDiodeReading
S1_sunGlintAngle
S1_sampleHeader_blanking
S1_sampleHeader_earthViewFirstSample
S1_sampleHeader_sampleNumber
S1_sampleHeader_tachSeconds
S1_sampleHeader_tachMicroSeconds
S1_navigation_scPos
S1_navigation_timeMidScanOffset
S1_navigation_scAttPitchGeod
S1_navigation_scVel
S1_navigation_scHeadingGround
S1_navigation_scAttRollGeod
S1_navigation_scAttRollGeoc
S1_navigation_scAlt
S1_navigation_scHeadingOrbital
S1_navigation_scLon
S1_navigation_scAttYawGeoc
S1_navigation_scAttPitchGeoc
S1_navigation_dprAlt
S1_navigation_greenHourAng
S1_navigation_sc

S2_cal2_TEMP_183GHZ_LO
S2_cal2_MR_LR_LEFT_TEMP
S2_cal2_TEMP_CALRES_4
S2_cal2_TEMP_V166GHZMXR
S2_cal2_onOrbitDiodeExcessTemp
S2_cal2_TEMP_H89GHZMXR
S2_cal2_trayTemperatureCount
S2_cal2_MR_LR_RGHT_TEMP
S2_cal2_CSR_TEMP1
S2_cal2_TEMP_183GHZMXR
S2_cal2_WarmIntrusionToColdViewIndex
S2_cal2_CSR_TEMP2
S2_cal2_MR_ICA_TEMP
S2_cal2_RSST_NDIODE_ST
S2_cal2_TEMP_RS_MR1
S2_cal2_BATC_CALRES_2
S2_cal2_TEMP_CALRES_5
S2_sunLocalTime
S2_nav2_SCE_SELECTION
S2_nav2_SCE_RATE
S2_scanStatus_operationalMode
S2_scanStatus_modeStatus
S2_scanStatus_targetSelectionMidScan
S2_scanStatus_geoWarning
S2_scanStatus_pointingStatus
S2_scanStatus_geoError
S2_scanStatus_dataQuality
S2_scanStatus_acsModeMidScan
S2_scanStatus_missing
S2_scanStatus_FractionalGranuleNumber
S2_scanStatus_SCorientation
S2_incidenceAngle
S2_RFIFlag
S3_scanStatus_operationalMode
S3_scanStatus_modeStatus
S3_scanStatus_targetSelectionMidScan
S3_scanStatus_geoWarning
S3_scanStatus_pointingStatus
S3_scanStatus_geoError
S3_scanStatus_dataQuality
S3_sca

/FS/VER/piaNP
/FS/VER/piaNPrainFree
/FS/VER/sigmaZeroNPCorrected
/FS/navigation/dprAlt
/FS/navigation/greenHourAng
/FS/navigation/scAlt
/FS/navigation/scAttPitchGeoc
/FS/navigation/scAttPitchGeod
/FS/navigation/scAttRollGeoc
/FS/navigation/scAttRollGeod
/FS/navigation/scAttYawGeoc
/FS/navigation/scAttYawGeod
/FS/navigation/scHeadingGround
/FS/navigation/scHeadingOrbital
/FS/navigation/scLat
/FS/navigation/scLon
/FS/navigation/scPos
/FS/navigation/scVel
/FS/scanStatus/FractionalGranuleNumber
/FS/scanStatus/SCorientation
/FS/scanStatus/acsModeMidScan
/FS/scanStatus/dataQuality
/FS/scanStatus/dataWarning
/FS/scanStatus/geoError
/FS/scanStatus/geoWarning
/FS/scanStatus/limitErrorFlag
/FS/scanStatus/missing
/FS/scanStatus/modeStatus
/FS/scanStatus/operationalMode
/FS/scanStatus/pointingStatus
/FS/scanStatus/targetSelectionMidScan
/FS/sunLocalTime
/HS/CSF/binBBBottom
/HS/CSF/binBBPeak
/HS/CSF/binBBTop
/HS/CSF/binDFRmMLBottom
/HS/CSF/binDFRmMLTop
/HS/CSF/flagBB
/HS/CSF/flagHeavyIcePrecip
/HS/

/HS/navigation/scHeadingGround
/HS/navigation/scHeadingOrbital
/HS/navigation/scLat
/HS/navigation/scLon
/HS/navigation/scPos
/HS/navigation/scVel
/HS/navigation/timeMidScanOffset
/HS/scanStatus/FractionalGranuleNumber
/HS/scanStatus/SCorientation
/HS/scanStatus/acsModeMidScan
/HS/scanStatus/dataQuality
/HS/scanStatus/dataWarning
/HS/scanStatus/geoError
/HS/scanStatus/geoWarning
/HS/scanStatus/limitErrorFlag
/HS/scanStatus/missing
/HS/scanStatus/modeStatus
/HS/scanStatus/operationalMode
/HS/scanStatus/pointingStatus
/HS/scanStatus/targetSelectionMidScan
/FS/CSF/binBBBottom
/FS/CSF/binBBPeak
/FS/CSF/binBBTop
/FS/CSF/binDFRmMLBottom
/FS/CSF/binDFRmMLTop
/FS/CSF/flagBB
/FS/CSF/flagHeavyIcePrecip
/FS/CSF/flagShallowRain
/FS/CSF/heightBB
/FS/CSF/qualityBB
/FS/CSF/qualityTypePrecip
/FS/CSF/typePrecip
/FS/CSF/widthBB
/FS/CSF/binHeavyIcePrecipBottom
/FS/CSF/binHeavyIcePrecipTop
/FS/CSF/nHeavyIcePrecip
/FS/DSD/binNode
/FS/DSD/paramRDm
/FS/Experimental/flagSurfaceSnowfall
/FS/Experimental/precip

S1_ScanTime_DayOfMonth
S1_profileScale
S1_profileTemp2mIndex
S1_precip2ndTertial
S1_precip1stTertial
S1_SCstatus_SCorientation
S1_SCstatus_SClatitude
S1_SCstatus_SClongitude
S1_SCstatus_SCaltitude
S1_SCstatus_FractionalGranuleNumber
S1_iceWaterPath
S1_sunGlintAngle
S1_surfacePrecipitation
S1_frozenPrecipitation
S1_cloudWaterPath
S1_totalColumnWaterVaporIndex
S1_probabilityOfPrecip
S1_temp2mIndex
S1_mostLikelyPrecipitation
S1_sunLocalTime
S1_surfaceTypeIndex
S1_rainWaterPath
S1_convectivePrecipitation
S1_L1CqualityFlag
S1_profileNumber
S1_airmassLiftIndex
S1_Longitude
S1_Latitude

The OPeNDAP service supports variable subsetting for GPM_3GPROFGPMGMI_DAY_CLIM_07
Variable names are:
InputFileNames
InputAlgorithmVersions
latentHeating
fractionQuality2
lat_bnds
fractionQuality1
iceWaterPath
surfacePrecipitation
frozenPrecipitation
fractionQuality0
npixTotal
cloudWaterPath
npixPrecipitation
snow
rainWater
cloudWater
surfaceTypeIndex
fractionQuality3
rainWaterPath
convectivePrecipitation
lon_

S1_ScanTime_DayOfYear
S1_ScanTime_Hour
S1_ScanTime_Month
S1_ScanTime_DayOfMonth
S1_profileScale
S1_profileTemp2mIndex
S1_precip2ndTertial
S1_precip1stTertial
S1_SCstatus_SCorientation
S1_SCstatus_SClatitude
S1_SCstatus_SClongitude
S1_SCstatus_SCaltitude
S1_SCstatus_FractionalGranuleNumber
S1_iceWaterPath
S1_sunGlintAngle
S1_surfacePrecipitation
S1_frozenPrecipitation
S1_cloudWaterPath
S1_totalColumnWaterVaporIndex
S1_probabilityOfPrecip
S1_temp2mIndex
S1_mostLikelyPrecipitation
S1_sunLocalTime
S1_surfaceTypeIndex
S1_rainWaterPath
S1_convectivePrecipitation
S1_L1CqualityFlag
S1_profileNumber
S1_airmassLiftIndex
S1_Longitude
S1_Latitude

The OPeNDAP service supports variable subsetting for GPM_3GPROFF17SSMIS_DAY_05
Variable names are:
fractionQuality2
cloudIce
fractionQuality1
iceWaterPath
surfacePrecipitation
frozenPrecipitation
fractionQuality0
npixTotal
cloudWaterPath
npixPrecipitation
snow
rainWater
cloudWater
surfaceTypeIndex
fractionQuality3
rainWaterPath
convectivePrecipitation
nl

S1_ScanTime_SecondOfDay
S1_ScanTime_Second
S1_ScanTime_DayOfYear
S1_ScanTime_Hour
S1_ScanTime_Month
S1_ScanTime_DayOfMonth
S1_SCstatus_SCorientation
S1_SCstatus_SClatitude
S1_SCstatus_SClongitude
S1_SCstatus_SCaltitude
S1_SCstatus_FractionalGranuleNumber
S1_sunGlintAngle
S1_incidenceAngleIndex
S1_Tc
S1_sunLocalTime
S1_Quality
S1_incidenceAngle
S2_ScanTime_MilliSecond
S2_ScanTime_Minute
S2_ScanTime_Year
S2_ScanTime_SecondOfDay
S2_ScanTime_Second
S2_ScanTime_DayOfYear
S2_ScanTime_Hour
S2_ScanTime_Month
S2_ScanTime_DayOfMonth
S2_SCstatus_SCorientation
S2_SCstatus_SClatitude
S2_SCstatus_SClongitude
S2_SCstatus_SCaltitude
S2_SCstatus_FractionalGranuleNumber
S2_sunGlintAngle
S2_incidenceAngleIndex
S2_Tc
S2_sunLocalTime
S2_Quality
S2_incidenceAngle
S3_ScanTime_MilliSecond
S3_ScanTime_Minute
S3_ScanTime_Year
S3_ScanTime_SecondOfDay
S3_ScanTime_Second
S3_ScanTime_DayOfYear
S3_ScanTime_Hour
S3_ScanTime_Month
S3_ScanTime_DayOfMonth
S3_SCstatus_SCorientation
S3_SCstatus_SClatitude
S3_SCstatus_SClo

KeyError: 'subset'

In [18]:
# Report on the results: DatasetID and the available services and their capabilities
for item in response['result']['items']:
    print("\n",item['dataset']['id'])
    # Check for subset services
    if item['services']['subset']: 
        for ss in item['services']['subset']:
            # make sure variable subsetting is supported
            print("  ",ss['agentConfig']['agentId'],ss['capabilities'])


 GPM_1BGMI_07
   OpenSearch ['temporal', 'spatial', 'format', 'service']
   OPeNDAP ['temporal', 'format', 'spatial', 'variables', 'dimensions', 'service']

 GPM_1AGMI_07
   OpenSearch ['temporal', 'spatial', 'format', 'service']
   OPeNDAP ['temporal', 'format', 'spatial', 'variables', 'dimensions', 'service']

 GPM_BASEGPMGMI_07
   OpenSearch ['temporal', 'spatial', 'format', 'service']
   OPeNDAP ['temporal', 'format', 'spatial', 'variables', 'dimensions', 'service']

 GPM_2HSLH_07
   OpenSearch ['temporal', 'spatial', 'format', 'service']
   OPeNDAP ['temporal', 'format', 'spatial', 'variables', 'dimensions', 'service']
   SUBSET_LEVEL2 ['temporal', 'format', 'spatial', 'dimensions', 'variables', 'diurnal', 'presentation', 'service']

 GPM_2HCSH_07
   OpenSearch ['temporal', 'spatial', 'format', 'service']
   OPeNDAP ['temporal', 'format', 'spatial', 'variables', 'dimensions', 'service']

 GPM_3GCSH_07
   OpenSearch ['temporal', 'spatial', 'format', 'service']
   OPeNDAP ['tempora

KeyError: 'subset'

In [22]:
# Report on the results: DatasetID for collections supported by SUBSET_LEVEL2
gpmL = []
for item in response['result']['items']:
    # Check for subset services
    try:
        if item['services']['subset']: 
            for ss in item['services']['subset']:
                # Check for SUBSET_LEVEL2
                if ss['agentConfig']['agentId'] == "SUBSET_LEVEL2":
                    print(item['dataset']['id'])
                    gpmL.append(item['dataset']['id'])
    except:
        continue
print(gpmL)


GPM_2HSLH_07
GPM_2ADPRENV_07
GPM_2AKuENV_07
GPM_2AKaENV_07
GPM_2AGPROFMETOPAMHS_05
GPM_2AGPROFNOAA19MHS_05
GPM_2ADPR_07
GPM_2BCMB_07
GPM_2AKu_07
GPM_2AKa_07
GPM_2AGPROFF19SSMIS_05
GPM_2AGPROFF16SSMIS_05
GPM_2AGPROFF18SSMIS_05
GPM_2AGPROFNPPATMS_05
GPM_2AGPROFMETOPBMHS_05
GPM_2AGPROFNOAA18MHS_05
GPM_2AGPROFGCOMW1AMSR2_05
GPM_2APRPSMT1SAPHIR_06
['GPM_2HSLH_07', 'GPM_2ADPRENV_07', 'GPM_2AKuENV_07', 'GPM_2AKaENV_07', 'GPM_2AGPROFMETOPAMHS_05', 'GPM_2AGPROFNOAA19MHS_05', 'GPM_2ADPR_07', 'GPM_2BCMB_07', 'GPM_2AKu_07', 'GPM_2AKa_07', 'GPM_2AGPROFF19SSMIS_05', 'GPM_2AGPROFF16SSMIS_05', 'GPM_2AGPROFF18SSMIS_05', 'GPM_2AGPROFNPPATMS_05', 'GPM_2AGPROFMETOPBMHS_05', 'GPM_2AGPROFNOAA18MHS_05', 'GPM_2AGPROFGCOMW1AMSR2_05', 'GPM_2APRPSMT1SAPHIR_06']


The complete response can be written to a local text file in case the entire metadata record is needed. 

In [11]:
# Write out the complete results to a file
fname='my_search_results.txt'
f = open(fname,'w')
f.write(json.dumps(response, indent=2, sort_keys=True))
f.close()
print('Complete metadata for each search result item has been written out to '+fname)

Complete metadata for each search result item has been written out to my_search_results.txt


**Additional Info:**  
[Complete reference documentation on the GES DISC Dataset Service API](https://disc.gsfc.nasa.gov/service/datasets)  
[How to Use the Web Services API for Subsetting](https://disc.gsfc.nasa.gov/information/howto?keywords=API&title=How%20to%20Use%20the%20Web%20Services%20API%20for%20Subsetting)  
[How to Use the Web Services API for Subsetting MERRA-2 Data](https://disc.gsfc.nasa.gov/information/howto?keywords=api&title=How%20to%20Use%20the%20Web%20Services%20API%20for%20Subsetting%20MERRA-2%20Data)  
[The GES DISC guide to data access](https://disc.gsfc.nasa.gov/data-access)

<font size="1">THE SUBJECT FILE IS PROVIDED "AS IS" WITHOUT ANY WARRANTY OF ANY KIND, EITHER EXPRESSED, IMPLIED, OR STATUTORY, INCLUDING, BUT NOT LIMITED TO, ANY WARRANTY THAT THE SUBJECT FILE WILL CONFORM TO SPECIFICATIONS, ANY IMPLIED WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE, OR FREEDOM FROM INFRINGEMENT, ANY WARRANTY THAT THE SUBJECT FILE WILL BE ERROR FREE, OR ANY WARRANTY THAT DOCUMENTATION, IF PROVIDED, WILL CONFORM TO THE SUBJECT FILE. THIS AGREEMENT DOES NOT, IN ANY MANNER, CONSTITUTE AN ENDORSEMENT BY GOVERNMENT AGENCY OR ANY PRIOR RECIPIENT OF ANY RESULTS, RESULTING DESIGNS, HARDWARE, SOFTWARE PRODUCTS OR ANY OTHER APPLICATIONS RESULTING FROM USE OF THE SUBJECT FILE. FURTHER, GOVERNMENT AGENCY DISCLAIMS ALL WARRANTIES AND LIABILITIES REGARDING THIRD-PARTY SOFTWARE, IF PRESENT IN THE SUBJECT FILE, AND DISTRIBUTES IT "AS IS."</font>